### Bayes Theorem: 

- We need this theorem when we are lacking a complete dataset, or to go through and make updates. 


$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$

#### The Cookie Problem: 

Two bowls of cookies:
- 1) 30 V, 10 C
- 2) 20 V, 20 C

Supposed you chose a bowl at random and a cookie at random. If the cookie is vanilla, what is the probability it came from Bowl 1? 

#### What Info Do We Have? 

- We want to know $P(A|B)$ -> Given we picked a Vanilla cookie, what is the likelihood our bowl was `#1`

- $P(A)$ = 0.5 -> equal chance between each bowl
- $P(B|A)$ = 0.75 -> given you chose from Bowl 1, probability of choosing a vanilla
- $P(B)$ = 5/8 -> overall likelihood of choosing a vanilla cookie. Our normalizing constant. 
    - Note: This should really be thought of as
    - $P(V) = P(B1) P(V|B1) + P(B2) P(V|B2)$
        - = 0.5 * (.75) + 0.5 * (0.5) 
        - = 0.25 + 0.375 = 0.625 (or 5/8)
    

In [1]:
p_a_given_b = (0.5 * 0.75) / (0.625)
print(p_a_given_b)

0.6


### Diachronic Bayes: Bayesian Update

- diachronic: related to change over time; the probability of the hypotheses will change as new data is discovered over time. 

How this works with Bayes:
- We can update the probability of `H` given some body of data `D`


$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$

Definitions:
- P(H) = probability of hypothesis prior to data -> the prior
    - In the cookie problem this was easy sinc each bowl has an equal chance of being chosen
    - Often we must use background info for a more subjective prior
- P(H|D) = probability of hypothesis after seeing data -> the posterior
- P(D|H) = the probability of the data under the hypothesis -> the likelihood
    - usually pretty straightforward 
- P(D) -> the total probability of the data regardless of hypothesis (e.g. Likelihood of vanilla cookies regardless of bowl)

Aligning with MECE helps:
- ME: mutually exclusive, so only one can be true
- CE: collectively exhaustive, so one must be true 

### Using Bayes Tables

- Helpful for handling updates

In [2]:
import pandas as pd 

# build table of prior and likelihood
table = pd.DataFrame(index=['Bowl 1', 'Bowl 2'])
table['prior'] = 1/2, 1/2
table['likelihood'] = 3/4, 1/2

# build the unnorm posterior (still need to divide by sum of unnorm)
table['unnorm'] = table['prior'] * table['likelihood']

# divide each "unnormed posterior" by the sum of unnormed posteriors
table['posterior'] = table['unnorm'] / table['unnorm'].sum()
table

,prior,likelihood,unnorm,posterior
Bowl 1,0.5,0.75,0.375,0.6
Bowl 2,0.5,0.50,0.250,0.4


### Bayes Table and 3 Hypotheses: 

Suppose I have a box with a 6-sided die, an 8-sided die, and a 12-sided die. I choose one of the dice at random, roll it, and report that the outcome is a 1. What is the probability that I chose the 6-sided die?

In [3]:
from fractions import Fraction

table2 = pd.DataFrame(index=[6, 8, 12])

# prior obviously 1/3
table2['prior'] = Fraction(1, 3)

# likelihood is likelihood of us seeing the data given hyp
# in our case this is easy as it is 1 / # of sides
table2['likelihood'] = Fraction(1, 6), Fraction(1, 8), Fraction(1, 12)
table2

,prior,likelihood
6,1/3,1/6
8,1/3,1/8
12,1/3,1/12


In [4]:
# Note: not a fan of this table being updated this way
def update(table):
    """Compute the posterior probabilities."""
    
    # solve unnormed posterior
    table['unnorm'] = table['prior'] * table['likelihood']
    
    # solve for probability of this data
    prob_data = table['unnorm'].sum()
    
    # build normed posterior 
    table['posterior'] = table['unnorm'] / prob_data
    
    return prob_data

In [5]:
prob_data = update(table2)
print(prob_data)
table2

1/8


,prior,likelihood,unnorm,posterior
6,1/3,1/6,1/18,4/9
8,1/3,1/8,1/24,1/3
12,1/3,1/12,1/36,2/9


#### Intuition:

- We don't have a ton of info yet, and since our priors are the same they aren't informative. 
- Since the six sided die has the highest likelihood of producing the outcome we saw it is the most likely. 

### Monty Hall Problemm

This always melts my mind, so I like thinking about the information we are being given as doors are opened from a larger problem.

If Monty had 100 doors and I pick one. Monty then removes 98 of the doors, leaving you with your door and another one. One of them has a prize. 
- It is easier to appreciate all of the information given here. The difference is Monty removes 98 doors for us and there is no change to our original probability, which is just 1%. 
- We know the sum of probs is 1, so monty's door has a 99% chance


#### Our Example:

Suppose you pick Door 1. Before opening the door you chose, Monty opens Door 3 and reveals a goat. Then Monty offers you the option to stick with your original choice or switch to the remaining unopened door.
- In this case, Monty is revealing to us that Door 3 has a goat. This is new information. 
- This means our set of possible options is now Door 1 or Door 2. Now door 1 still has a `1/3` likelihood, but door 2 will be `2/3` given it is just `1 - 1/3`

In our problem, we want to know:
- P(Door 1 Has Car | Door 3 Goat)
- P(Door 2 Has Car | Door 3 Goat)
- P(Door 3 Has Car | Door 3 Goat)

In [6]:
# here is our starting point
table3 = pd.DataFrame(index=['Door 1', 'Door 2', 'Door 3'])
table3['prior'] = Fraction(1, 3)
table3

,prior
Door 1,1/3
Door 2,1/3
Door 3,1/3


We can now think about how Monty will act based on the hypothesis: 

- If the car is behind Door 1, Monty chooses Door 2 or 3 at random, so the probability he opens Door 3 is 1/2.
    - P(Door 3 Goat| Car Door 1) 

- If the car is behind Door 2, Monty has to open Door 3, so the probability of the data under this hypothesis is 1.
    - P(Door 3 Goat| Car Door 2) 

- If the car is behind Door 3, Monty does not open it, so the probability of the data under this hypothesis is 0.
    - P(Door 3 Goat| Car Door 3) 
    
    
There is a lot of information embedded in this action: 
- If he opens Door 3 we know th ecar is not behind Door 3. 
- Opening Door 3 is more likely if the car is behind Door 2, less likely if behind Door 1. 
    - He can't open Door 2 is Door 3 holds the car. If Door 1 holds the car, it is a random selection. 

In [7]:
table3['likelihood'] = Fraction(1, 2), 1, 0
table3

,prior,likelihood
Door 1,1/3,1/2
Door 2,1/3,1
Door 3,1/3,0


In [8]:
update(table3)
table3

,prior,likelihood,unnorm,posterior
Door 1,1/3,1/2,1/6,1/3
Door 2,1/3,1,1/3,2/3
Door 3,1/3,0,0,0


### Summary